# Ollama + OpenAI + Python

## 1. Specifică numele modelului

Dacă ai utilizat un model diferit de "phi3:mini", modifică valoarea în celula de mai jos. Acea variabilă va fi folosită în cod pe tot parcursul notebook-ului.


In [ ]:
MODEL_NAME = "phi3:mini"

## 2. Configurarea clientului Open AI

De obicei, clientul OpenAI este utilizat cu OpenAI.com sau Azure OpenAI pentru a interacționa cu modele lingvistice de mari dimensiuni.  
Cu toate acestea, poate fi folosit și cu Ollama, deoarece Ollama oferă un endpoint compatibil cu OpenAI la "http://localhost:11434/v1".


In [ ]:
%pip install openai

In [ ]:
import openai

client = openai.OpenAI(
    base_url="http://localhost:11434/v1",
    api_key="nokeyneeded",
)

## 3. Generați un răspuns pentru o conversație

Acum putem folosi SDK-ul OpenAI pentru a genera un răspuns pentru o conversație. Această cerere ar trebui să genereze un haiku despre pisici:


In [ ]:
response = client.chat.completions.create(
    model=MODEL_NAME,
    temperature=0.7,
    n=1,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Write a haiku about a hungry cat"},
    ],
)

print("Response:")
print(response.choices[0].message.content)


## 4. Ingineria prompturilor

Primul mesaj trimis modelului lingvistic este numit "mesajul sistemului" sau "promptul sistemului" și stabilește instrucțiunile generale pentru model.  
Poți furniza propriul tău prompt personalizat pentru sistem, pentru a ghida modelul lingvistic să genereze rezultate într-un mod diferit.  
Modifică `SYSTEM_MESSAGE` de mai jos pentru a răspunde ca personajul tău preferat dintr-un film/serial celebru sau inspiră-te pentru alte prompturi de sistem din [Awesome ChatGPT Prompts](https://github.com/f/awesome-chatgpt-prompts?tab=readme-ov-file#prompts).

După ce ai personalizat mesajul sistemului, oferă prima întrebare a utilizatorului în `USER_MESSAGE`.


In [ ]:
SYSTEM_MESSAGE = """
I want you to act like Elmo from Sesame Street.
I want you to respond and answer like Elmo using the tone, manner and vocabulary that Elmo would use.
Do not write any explanations. Only answer like Elmo.
You must know all of the knowledge of Elmo, and nothing more.
"""

USER_MESSAGE = """
Hi Elmo, how are you doing today?
"""

response = client.chat.completions.create(
    model=MODEL_NAME,
    temperature=0.7,
    n=1,
    messages=[
        {"role": "system", "content": SYSTEM_MESSAGE},
        {"role": "user", "content": USER_MESSAGE},
    ],
)

print("Response:")
print(response.choices[0].message.content)


## 5. Exemple cu puține instanțe

O altă modalitate de a ghida un model lingvistic este să oferi "puține instanțe", o secvență de întrebări/răspunsuri exemplu care demonstrează cum ar trebui să răspundă.

Exemplul de mai jos încearcă să facă un model lingvistic să se comporte ca un asistent didactic, oferind câteva exemple de întrebări și răspunsuri pe care un asistent didactic le-ar putea da, iar apoi solicită modelului o întrebare pe care un student ar putea să o pună.

Încearcă mai întâi, apoi modifică `SYSTEM_MESSAGE`, `EXAMPLES` și `USER_MESSAGE` pentru un nou scenariu.


In [ ]:
SYSTEM_MESSAGE = """
You are a helpful assistant that helps students with their homework.
Instead of providing the full answer, you respond with a hint or a clue.
"""

EXAMPLES = [
    (
        "What is the capital of France?",
        "Can you remember the name of the city that is known for the Eiffel Tower?"
    ),
    (
        "What is the square root of 144?",
        "What number multiplied by itself equals 144?"
    ),
    (   "What is the atomic number of oxygen?",
        "How many protons does an oxygen atom have?"
    ),
]

USER_MESSAGE = "What is the largest planet in our solar system?"


response = client.chat.completions.create(
    model=MODEL_NAME,
    temperature=0.7,
    n=1,
    messages=[
        {"role": "system", "content": SYSTEM_MESSAGE},
        {"role": "user", "content": EXAMPLES[0][0]},
        {"role": "assistant", "content": EXAMPLES[0][1]},
        {"role": "user", "content": EXAMPLES[1][0]},
        {"role": "assistant", "content": EXAMPLES[1][1]},
        {"role": "user", "content": EXAMPLES[2][0]},
        {"role": "assistant", "content": EXAMPLES[2][1]},
        {"role": "user", "content": USER_MESSAGE},
    ],
)


print("Response:")
print(response.choices[0].message.content)

## 6. Generarea Augmentată prin Recuperare

RAG (Generarea Augmentată prin Recuperare) este o tehnică prin care un model lingvistic poate răspunde cu acuratețe la întrebări dintr-un anumit domeniu, prin recuperarea inițială a informațiilor relevante dintr-o sursă de cunoștințe și apoi generarea unui răspuns bazat pe aceste informații.

Am furnizat un fișier CSV local cu date despre mașinile hibride. Codul de mai jos citește fișierul CSV, caută potriviri pentru întrebarea utilizatorului și apoi generează un răspuns bazat pe informațiile găsite. Rețineți că acest proces va dura mai mult decât oricare dintre exemplele anterioare, deoarece trimite mai multe date către model. Dacă observați că răspunsul nu este încă bazat pe datele furnizate, puteți încerca ingineria sistemului sau alte modele. În general, RAG este mai eficient fie cu modele mai mari, fie cu versiuni ajustate ale SLM-urilor.


In [ ]:
import csv

SYSTEM_MESSAGE = """
You are a helpful assistant that answers questions about cars based off a hybrid car data set.
You must use the data set to answer the questions, you should not provide any information that is not in the provided sources.
"""

USER_MESSAGE = "how fast is a prius?"

# Open the CSV and store in a list
with open("hybrid.csv", "r") as file:
    reader = csv.reader(file)
    rows = list(reader)

# Normalize the user question to replace punctuation and make lowercase
normalized_message = USER_MESSAGE.lower().replace("?", "").replace("(", " ").replace(")", " ")

# Search the CSV for user question using very naive search
words = normalized_message.split()
matches = []
for row in rows[1:]:
    # if the word matches any word in row, add the row to the matches
    if any(word in row[0].lower().split() for word in words) or any(word in row[5].lower().split() for word in words):
        matches.append(row)

# Format as a markdown table, since language models understand markdown
matches_table = " | ".join(rows[0]) + "\n" + " | ".join(" --- " for _ in range(len(rows[0]))) + "\n"
matches_table += "\n".join(" | ".join(row) for row in matches)
print(f"Found {len(matches)} matches:")
print(matches_table)

# Now we can use the matches to generate a response
response = client.chat.completions.create(
    model=MODEL_NAME,
    temperature=0.7,
    n=1,
    messages=[
        {"role": "system", "content": SYSTEM_MESSAGE},
        {"role": "user", "content": USER_MESSAGE + "\nSources: " + matches_table},
    ],
)

print("Response:")
print(response.choices[0].message.content)


---

**Declinare de responsabilitate**:  
Acest document a fost tradus folosind serviciul de traducere AI [Co-op Translator](https://github.com/Azure/co-op-translator). Deși ne străduim să asigurăm acuratețea, vă rugăm să rețineți că traducerile automate pot conține erori sau inexactități. Documentul original în limba sa natală ar trebui considerat sursa autoritară. Pentru informații critice, se recomandă traducerea profesională realizată de un specialist uman. Nu ne asumăm responsabilitatea pentru eventualele neînțelegeri sau interpretări greșite care pot apărea din utilizarea acestei traduceri.
